In [1]:
from sklearn.feature_selection import mutual_info_regression, f_regression
from sklearn.feature_selection import SelectPercentile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from subprocess import check_output


In [3]:
#df = pd.read_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/CNTYVARS.csv')
df = pd.read_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\CNTYVARS.csv')
df['HSGRAD_PRCNT'] = df['HSGRAD_CY']/df['TOTPOP_CY']

In [4]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
print(dtype_df[dtype_df['Column Type'] == 'object'])

         Count Column Type
783   TSEGCODE      object
784   TSEGNAME      object
854  TLIFECODE      object
855  TLIFENAME      object


In [5]:
dp = ['OBJECTID','Customers', 'UNITS', 'SALES', 'LOG_UNITS', 'TSEGCODE','TSEGNAME','TLIFECODE','TLIFENAME']

X = df.drop(dp, axis=1)

#X2 = transformer.fit_transform(X, y)

#X2.head()

In [6]:
y = df[['LOG_UNITS']]
y.shape

(3109, 1)

In [29]:
selector = SelectPercentile(f_regression, percentile=20)
columns = X.columns
selector.fit_transform(X, np.array(y))
labels = [columns[x] for x in selector.get_support(indices=True) if x]
XNew = pd.DataFrame(selector.fit_transform(X, np.array(y)), columns = labels)
columns = XNew.columns


lst = list()
for n, s in zip(columns, selector.scores_):
    lst.append((s, n))

scores = pd.DataFrame(lst, columns=['Score','Name'])
    


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [30]:
inds = list()
for val in columns:
    #print(val)
    inds.append((val, XNew.columns.get_loc(val)))
    
indices = pd.DataFrame(inds, columns=['Name','XNew_Index'])
    


In [31]:
scores.sort_values(by='Score',inplace=True, ascending=True)
scores.reset_index(inplace=True)
#scores.to_csv(r'C:\Users\pairwin\Documents\GitHub\IPy_Notebooks\SQL\varscores.csv')
#scores.to_csv(r'/home/pirwin/Git/IPy_Notebooks/SQL/varscores.csv')

In [32]:
scores.drop('index', inplace=True, axis = 1)

df = pd.merge(scores, indices, how='inner',left_on='Name',right_on='Name')



In [33]:


def calculate_vif(X, thresh=7.5):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            # Loop repeatedly until we find that all columns within our dataset
            # have a VIF value we're happy with.
            variables = X.columns
            dropped=False
            vif = []
            new_vif = 0
            for var in variables:
                new_vif = variance_inflation_factor(X.as_matrix(), X.columns.get_loc(var))
                vif.append(new_vif)
                if np.isinf(new_vif):
                    break
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                #df = df[df['Name'] != str(X.columns[maxloc])]
                dropped=True
        return X
    
finaldf = calculate_vif(XNew)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Dropping POP60_CY with vif=inf
Dropping POP65_CY with vif=inf
Dropping POP70_CY with vif=inf
Dropping POP75_CY with vif=inf
Dropping POP80_CY with vif=inf
Dropping MALE60_CY with vif=inf
Dropping MALE65_CY with vif=inf
Dropping MALE70_CY with vif=inf
Dropping MALE75_CY with vif=inf
Dropping MALE80_CY with vif=inf
Dropping FEM60_CY with vif=inf
Dropping FEM65_CY with vif=inf
Dropping FEM70_CY with vif=inf
Dropping FEM75_CY with vif=inf
Dropping FEM80_CY with vif=inf
Dropping AGE58_CY with vif=inf
Dropping AGE59_CY with vif=inf
Dropping AGE60_CY with vif=inf
Dropping AGE61_CY with vif=inf
Dropping AGE62_CY with vif=inf
Dropping AGE63_CY with vif=inf
Dropping AGE64_CY with vif=inf
Dropping AGE65_CY with vif=inf
Dropping AGE66_CY with vif=inf
Dropping AGE67_CY with vif=inf
Dropping AGE68_CY with vif=inf
Dropping AGE69_CY with vif=inf
Dropping AGE70_CY with vif=inf
Dropping AGE71_CY with vif=inf
Dropping AGE72_CY with vif=inf
Dropping AGE73_CY with vif=inf
Dropping AGE74_CY with vif=inf
Dro

Dropping A65DI25_CY with vif=94.55575751181497
Dropping NHSPWHT_CY with vif=87.3042463056741
Dropping S10_SALES with vif=83.42918105391244
Dropping S07_BUS with vif=70.55125798000711
Dropping ASSCDEG_CY with vif=68.02361151528055
Dropping N18_SALES with vif=67.11072341061796
Dropping VAL150K_CY with vif=63.04939362768436
Dropping N09_BUS with vif=61.118862670093456
Dropping A55DI25_CY with vif=58.48534938695471
Dropping A25NW150CY with vif=51.27744558263766
Dropping S03_EMP with vif=45.53606446417483
Dropping A75DI15_CY with vif=41.415385931598365
Dropping N13_EMP with vif=26.68134444977266
Dropping GED_CY with vif=25.029237495197147
Dropping AGGNWA55CY with vif=23.02159747653444
Dropping A15NW35_CY with vif=21.605432093256937
Dropping N12_EMP with vif=21.586376596797745
Dropping N09_EMP with vif=17.867541109382685
Dropping S31_BUS with vif=15.281064840908478
Dropping N33_EMP with vif=15.1587870543486
Dropping S02_BUS with vif=14.561249018314431
Dropping INDUTIL_CY with vif=14.02513264

In [34]:
finaldf.tail()

,INDPUBL_CY,VAL0_CY,VAL100K_CY,THHGRPL4,THHGRPL5,THHGRPL6,THHGRPU5,THHGRPU6,TADULT21,TADULT38,S31_EMP,N04_BUS,N36_BUS
3104,191.0,346.0,419.0,0.0,0.0,1944.0,992.0,1871.0,0.0,0.0,282.0,1.0,7.0
3105,2454.0,1081.0,1807.0,3647.0,3310.0,8228.0,4306.0,7802.0,9487.0,737.0,1630.0,7.0,32.0
3106,248.0,137.0,234.0,0.0,0.0,2093.0,0.0,2598.0,0.0,0.0,293.0,1.0,23.0
3107,423.0,430.0,788.0,855.0,4080.0,2032.0,1348.0,3336.0,0.0,1555.0,722.0,9.0,57.0
3108,778.0,621.0,1855.0,0.0,0.0,7934.0,3952.0,7939.0,0.0,0.0,1566.0,5.0,33.0
